In [1]:
import requests

In [2]:
def get_article_revisions(title):
    revisions = []

    # create a base url for the api and then a normal url which is initially
    # just a copy of it
    # The following line is what the requests call is doing, basically.
    # f"http://en.wikipedia.org/w/api.php/?action=query&titles={title}&prop=revisions&rvprop=flags|timestamp|user|size|ids&rvlimit=500&format=json&continue="
    # e.g.: http://en.wikipedia.org/w/api.php/?action=query&titles=Harry_Potter&prop=revisions&rvprop=flags|timestamp|user|size|ids&rvlimit=500&format=json&continue=
    wp_api_url = "http://en.wikipedia.org/w/api.php/"

    parameters = {'action' : 'query',
                  'titles' : title,
                  'prop' : 'revisions',
                  'rvprop' : 'flags|timestamp|user|size|ids',
                  'rvlimit' : 500,
                  'format' : 'json',
                  'continue' : '' }

    # we'll repeat this forever (i.e., we'll only stop when we find
    # the "break" command)
    while True:
        # the first line open the urls but also handles unicode urls
        call = requests.get(wp_api_url, params=parameters)
        api_answer = call.json()

        # get the list of pages from the json object
        pages = api_answer["query"]["pages"]

        # for every page, (there should always be only one) get its revisions:
        for page in pages:
            query_revisions = pages[page]["revisions"]

            # for every revision, first we do some cleaning up
            for rev in query_revisions:
                #print(rev)
                # let's continue/skip this revision if the user is hidden
                if "userhidden" in rev:
                    continue
                
                # 1: add a title field for the article because we're going to mix them together
                rev["title"] = title

                # 2: let's "recode" anon so it's true or false instead of present/missing
                if "anon" in rev:
                    rev["anon"] = True
                else:
                    rev["anon"] = False

                # 3: let's recode "minor" in the same way
                if "minor" in rev:
                    rev["minor"] = True
                else:
                    rev["minor"] = False

                # we're going to change the timestamp to make it work a little better in excel/spreadsheets
                rev["timestamp"] = rev["timestamp"].replace("T", " ")
                rev["timestamp"] = rev["timestamp"].replace("Z", "")

                # finally, save the revisions we've seen to a varaible
                revisions.append(rev)

        # 'continue' tells us there's more revisions to add
        if 'continue' in api_answer:
            # replace the 'continue' parameter with the contents of the
            # api_answer dictionary.
            parameters.update(api_answer['continue'])
        else:
            break

    # return all the revisions for this page
    return(revisions)


In [3]:
category = "Harry Potter"

# we'll use another api called petscan to grab a list of pages in
# categories and subcategories. it works like all the other apis we've
# studied!
#
# The following requests call basically does the same thing as this string:
# f"https://petscan.wmflabs.org/?depth=10&categories={category}&format=json&doit=1"
url_petscan = "https://petscan.wmflabs.org/"

parameters = {'depth' : 10,
              'categories' : category,
              'format' : 'json',
              'doit' : 1}

# r = requests.get("https://petscan.wmflabs.org/?depth=10&categories=Harry Potter&format=json&doit=1")

In [4]:
r = requests.get(url_petscan, params=parameters)

In [9]:
articles_json = r.json()
# this gets us to a list and no longer a dictionary
articles = articles_json["*"][0]["a"]["*"]

In [6]:
# open a file to print the header
output_file = open("hp_wiki.tsv", "w", encoding='utf-8')
print("\t".join(["title", "user", "timestamp", "size", "anon", "minor", "revid"]), file=output_file)

In [7]:
# for every article
for article in articles:
    # skip this until it's an article
    if article["namespace"] != 0:
        continue

    # first grab the article's title
    title = article["title"]
    print(title)

    # get the list of revisions from our function and then iterate through it,
    # printing it to our output file
    revisions = get_article_revisions(title)
    for rev in revisions:
        print("\t".join([rev["title"], rev["user"], rev["timestamp"],
                         str(rev["size"]), str(rev["anon"]),
                         str(rev["minor"]), str(rev["revid"])]),
             file=output_file)

J._K._Rowling
Severus_Snape
Slytherin
Draco_Malfoy
Muggle
Gryffindor
Hermione_Granger
Lord_Voldemort
Hogwarts_Express
Albus_Dumbledore
Hogwarts
Sorting_Hat
Harry_Potter_and_the_Philosopher's_Stone
Nearly_Headless_Nick
Quidditch
Ravenclaw
Hufflepuff
Cedric_Diggory
Harry_Potter_and_the_Order_of_the_Phoenix
Defence_Against_the_Dark_Arts
Death_Eater
Molly_Weasley
Percy_Weasley
Fred_Weasley
Bill_Weasley
George_Weasley
Charlie_Weasley
Arthur_Weasley
Cho_Chang
Parvati_Patil
Harry_Potter_and_the_Chamber_of_Secrets
Arabella_Figg
Remus_Lupin
Fleur_Delacour
Narcissa_Malfoy
Mirror_of_Erised
Harry_Potter_in_translation
Neville_Longbottom
Dolores_Umbridge
Hedwig_(Harry_Potter)
Chamber_of_Secrets
Minerva_McGonagall
Sirius_Black
Harry_Potter_and_the_Prisoner_of_Azkaban
Wizengamot
Cornelius_Fudge
Pigwidgeon
Vernon_Dursley
Alastor_Moody
Nymphadora_Tonks
Order_of_the_Phoenix_(fictional_organisation)
Harry_Potter_and_the_Goblet_of_Fire
Pensieve
Tanya_Grotter
Gilderoy_Lockhart
Sibyll_Trelawney
Dementor
Ded

Silvanus_Kettleburn
Harry_Potter_and_the_Escape_from_Gringotts
Book_of_Potions
IQA_World_Cup_VI
Ministry_of_Magic_(band)
Wrockstock
Myles_Kane
British_Quidditch_Cup
QuidditchUK
IQA_World_Cup_III
IQA_World_Cup_V
US_Quidditch_Cup
Oxford_University_Quidditch_Club
The_Wizarding_World_of_Harry_Potter_(Universal_Studios_Japan)
Belgium_national_quidditch_team
Quidditch_Canada
Quidditch_Nederland
Quidditch_Benelux
United_Kingdom_national_quidditch_team
Belgian_Quidditch_Federation
Fantastic_Beasts:_The_Crimes_of_Grindelwald
US_Quidditch
Quidditch_in_Australia
Canada_national_quidditch_team
European_Quidditch_Cup
Pollomuhku_ja_Posityyhtynen
IQA_World_Cup
Bastian_Steel_Bukan_Cowok_Biasa
Quidditch_in_Canada
Amortentia
Draught_of_Living_Death
Harry_Potter_and_the_Cursed_Child
Hogwarts_Express_(Universal_Orlando_Resort)
European_Games_(quidditch)
2015_IQA_European_Games
Fantastic_Beasts_and_Where_to_Find_Them_(film)
Quidditch_in_Spain
2014_IQA_Global_Games
Spanish_Quidditch_Association
Spain_nation

In [8]:
# close the file, we're done here!
output_file.close()